## GET RELEVANT DATA FOR DATA QUALITY CONTROL

``` sql
SELECT
    creation_time,
    table_id,
    user_email,
    job_type,
    statement_type,
    start_time,
    end_time,
    query,
    total_bytes_processed,
    total_bytes_billed,
    total_slot_ms
FROM
`region-us`.INFORMATION_SCHEMA.JOBS as J,
UNNEST(referenced_tables) AS RT
WHERE RT.dataset_id in ('') -- REPLACE IT WITH THE LIST OF THE DATASETS FOR YOUR PROJECT
    AND J.project_id = '' -- REPLACE IT WITH THE NAME OF YOUT PROJECT
    AND J.state = 'DONE'
```

## GET THE LIST OF ALL THE TABLES WITHIN YOUR PROJECT

```sql
SELECT
    table_name
FROM YOUR_PROJECT.YOUR_DATASET.INFORMATION_SCHEMA.TABLES -- Replace YOUR_PROJECT and YOUR_DATASET for the correspondent names for your project. 
```

## TRUNCATE ALL TABLES FOR A SPECIFIC DATASET

```sql
DECLARE TABLE_NAME STRING;
DECLARE TABLE_SIZE INT64;
DECLARE X INT64 DEFAULT 0;
DECLARE STATEMENTS ARRAY<STRING>;

SET STATEMENTS = (
SELECT 
    ARRAY_AGG(CONCAT('TRUNCATE TABLE `', table_schema, '.', table_name, '`;'))
FROM YOUR_PROJECT.YOUR_DATASET.INFORMATION_SCHEMA.TABLES -- REPLACE BOTH YOUR_PROJECT AND YOUR_DATASET WITH THE CORRESPONDENT NAMES FOR YOUR PROJECT
WHERE table_type = 'BASE TABLE'
    AND table_schema = '' -- REPLACE IT WITH THE DATASET OF YOUR PROJECT
);

SET table_size = array_length(statements);

LOOP
SET X = X + 1;
IF x > table_size THEN
    LEAVE;
END IF;
EXECUTE IMMEDIATE STATEMENTS[ORDINAL(X)];
END LOOP;  
```

### CHECKING DATA SWAMPIFICATION TABLES

```sql
SELECT  
    DISTINCT table_id -- REPLACE WITH A COLUMN WHICH HAS THE TABLE NAME DATA
FROM YOUR_PROJECT.YOUR_DATASET.INFORMATION_SCHEMA.TABLES -- REPLACE BOTH YOUR_PROJECT AND YOUR_DATASET WITH THE CORRESPONDENT NAMES FOR YOUR PROJECT
WHERE creation_time > TIMESTAMP_SUB(CURRENT_TIMESTAMP() , INTERVAL 30 DAY) -- SHIFT TO A SUITABLE INTERVAL
EXCEPT DISTINCT
SELECT 
    DISTINCT table_id -- REPLACE WITH A COLUMN WHICH HAS THE TABLE NAME DATA
FROM YOUR_PROJECT.YOUR_DATASET.INFORMATION_SCHEMA.TABLES -- REPLACE BOTH YOUR_PROJECT AND YOUR_DATASET WITH THE CORRESPONDENT NAMES FOR YOUR PROJECT
WHERE creation_time > TIMESTAMP_SUB(CURRENT_TIMESTAMP() , INTERVAL 30 DAY) -- SHIFT TO A SUITABLE INTERVAL
```